### RNN 활용 분석
- 시계열 데이터라 생각하고 RNN 사용해보자
- 월단위까지밖에 없어서 괜찮을까? -> 제대로 학습 안돼도 경향성 있으면 어느 정도는 학습되는 부분이 있지 않을까? 생각
- 확실히 데이터 전처리 부분에서 아는게 많이 없는듯,, ㅠ
- 우선 데이터 시간순 정렬 -> 일자(feature)와 총전력량(label)만으로 가능한지 확인 
- github pr test...

In [ ]:
#설치 확인
import torch
print(torch.__version__)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

#### 📌 데이터 특성 확인하기

- 기본 인코딩으론 한글이 깨져서 euc-kr인코딩으로 변경함
- 근데... 같은 월이 여러개 있는데 이걸 시계열성으로 할 수가 있나?
- -> 기존 데이터에 랜덤으로 시간값을 추가해주려 했는데 이건 실패 (코드 오류&데이터 오염될 수 있다 판단)

In [1]:
#데이터 준비
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

train_df = pd.read_csv('TRAIN_DATA.csv', encoding='euc-kr')
test_df = pd.read_csv('TEST_DATA.csv', encoding='euc-kr')

#데이터 확인
print(train_df.head())
print(test_df.head())

FileNotFoundError: [Errno 2] No such file or directory: 'TRAIN_DATA.csv'

In [ ]:
print(test_df.columns)

In [ ]:
import pandas as pd

data = pd.read_csv('TRAIN_DATA.csv', encoding='euc-kr')

sorted_data = data.sort_values(by='DATA_YM')
extracted_data = sorted_data[['DATA_YM', 'TOTAL_ELEC']]

print(extracted_data.head())  #확인용

extracted_data.to_csv('TRAIN_DATA_sorted.csv', index=False, encoding='euc-kr')

In [ ]:
train_df = pd.read_csv('TRAIN_DATA_sorted.csv', encoding='euc-kr')

#필요한 변수 선택 및 전처리
#'TOTAL_ELEC'를 예측할 목표 변수로 설정하고 나머지 변수는 특징으로 설정
features = [
    'DATA_YM'
]

X = train_df[features].values  #특징값
y = train_df['TOTAL_ELEC'].values  #목표값

#데이터 정규화 (일반적으로 LSTM/GRU에 대해 좋은 성능을 보임)
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

#학습용 데이터와 검증용 데이터 분할
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

#테스트 데이터 전처리
X_test = test_df[features].values  #테스트 데이터 특징
X_test_scaled = scaler.transform(X_test)  #테스트 데이터 정규화

# PyTorch 텐서로 변환
X_train_tensor = torch.FloatTensor(X_train).unsqueeze(1)  # (배치, 시퀀스 길이, 특성 수)
y_train_tensor = torch.FloatTensor(y_train).view(-1, 1)  # y는 2D 텐서 (배치, 1)
X_val_tensor = torch.FloatTensor(X_val).unsqueeze(1)
y_val_tensor = torch.FloatTensor(y_val).view(-1, 1)
X_test_tensor = torch.FloatTensor(X_test_scaled).unsqueeze(1)

#데이터로더 생성
from torch.utils.data import DataLoader, TensorDataset

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

#RNN 모델 정의
class SingleRNN(nn.Module):
    def __init__(self, rnn_type, input_size, hidden_size, dropout=0, bidirectional=False):
        super(SingleRNN, self).__init__()
        
        self.rnn = getattr(nn, rnn_type)(input_size, hidden_size, num_layers=1, dropout=dropout, batch_first=True, bidirectional=bidirectional)
        self.fc = nn.Linear(hidden_size * (1 + int(bidirectional)), 1)  # Linear layer for output

    def forward(self, x):
        rnn_output, _ = self.rnn(x)
        # 최종 hidden state의 출력을 fc layer에 입력
        output = self.fc(rnn_output[:, -1, :])  # Get the last time step
        return output

#하이퍼파라미터 설정
input_size = 1  # 입력 벡터 크기 (feature 수)
hidden_size = 20  # 히든 레이어의 크기
dropout = 0.2
bidirectional = True  # Bidirectional RNN 사용 여부
rnn_type = 'LSTM'  # 사용할 RNN 타입

#모델 초기화
model = SingleRNN(rnn_type, input_size, hidden_size, dropout, bidirectional)

#손실 함수 및 옵티마이저 설정
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

#모델 학습
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:  # train_loader로부터 배치 가져오기
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    
    #Validation step
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for val_inputs, val_labels in val_loader:
            val_outputs = model(val_inputs)
            val_loss += criterion(val_outputs, val_labels).item()
    
    val_loss /= len(val_loader)  #평균 검증 손실
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Val Loss: {val_loss:.4f}')

#테스트 및 결과 출력
model.eval()
# test_data로부터의 예측 수행
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    # 예측 결과를 활용하여 후속 작업을 진행 (예: Excel 파일로 저장)

### 📌 검증 dataset에서 RMSE 계산

In [ ]:
import numpy as np

# 실제 값을 가져오는 방법 수정
actual_values = y_val 
predicted_values = model(X_val_tensor).detach().numpy()  # 예측 값 변환

# RMSE 계산
rmse = np.sqrt(np.mean((actual_values - predicted_values) ** 2))
print(f"RMSE: {rmse}")